# SESION 4 DICIEMBRE

# Modificamos columnas que eran object a float 

In [39]:
import pandas as pd
import numpy as np
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt

In [40]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [41]:
df_rawdata = pd.read_csv("raw_data_20231201.csv", index_col=0)

In [42]:
def cambiar_comas(cadena):
    try:
        return float(cadena.replace(",", "."))
    except:
        return np.nan
    
df_rawdata["Monthlyincome"] = df_rawdata["Monthlyincome"].apply(cambiar_comas)
df_rawdata["Performancerating"] = df_rawdata["Performancerating"].apply(cambiar_comas)
df_rawdata["Totalworkingyears"] = df_rawdata["Totalworkingyears"].apply(cambiar_comas)

In [43]:
df_rawdata["Employeenumber"]= df_rawdata["Employeenumber"].apply(cambiar_comas)

In [44]:
df_rawdata["Employeenumber"].isnull().sum()

431

# Gestión de nulos

Cambiamos los nulos de Employeenumber con un "auto-incremental"

In [45]:
# Encuentra el último ID existente
ultimo_id = df_rawdata['Employeenumber'].max()
# Define una variable para el siguiente ID después del último
siguiente_id = ultimo_id + 1
# Itera sobre los índices del DataFrame
for indice in df_rawdata.index:
    if pd.isnull(df_rawdata.at[indice, 'Employeenumber']):  # Verifica si el valor es nulo
        df_rawdata.at[indice, 'Employeenumber'] = siguiente_id  # Asigna el siguiente ID
        siguiente_id += 1  # Incrementa el siguiente ID
df_rawdata["Employeenumber"] = df_rawdata["Employeenumber"].astype(int)
# Muestra el DataFrame actualizado
print(df_rawdata["Employeenumber"].tail(10))

1604     927
1605     941
1606     956
1607    2497
1608     966
1609     967
1610     972
1611    2498
1612     990
1613    2499
Name: Employeenumber, dtype: int64


In [46]:
# Muestra el DataFrame actualizado
df_rawdata["Employeenumber"].dtype

dtype('int64')

In [47]:
nulls_cat = df_rawdata[df_rawdata.columns[df_rawdata.isnull().any()]].select_dtypes(include = "O").columns
nulls_cat

Index(['Businesstravel', 'Department', 'Educationfield', 'Maritalstatus',
       'Overtime'],
      dtype='object')

In [48]:
(df_rawdata[['Businesstravel', 'Department', 'Educationfield', 'Maritalstatus','Overtime']].isnull().sum() / df_rawdata.shape[0]) * 100

Businesstravel    47.880795
Department        81.258278
Educationfield    46.490066
Maritalstatus     40.264901
Overtime          41.721854
dtype: float64

In [49]:
# sacamos el 'value_counts()' de cada una de las columnas categóricas que tienen nulos para saber como es la distribución de sus categorías
for col in nulls_cat:
    print(f"La distribución de las categorías para la columna {col.upper()}")
    display(df_rawdata[col].value_counts() / df_rawdata.shape[0])
    print("........................")

La distribución de las categorías para la columna BUSINESSTRAVEL


travel_rarely        0.364901
travel_frequently    0.101325
non-travel           0.054967
Name: Businesstravel, dtype: float64

........................
La distribución de las categorías para la columna DEPARTMENT


 Research & Development     0.122517
 Sales                      0.055629
 Human Resources            0.009272
Name: Department, dtype: float64

........................
La distribución de las categorías para la columna EDUCATIONFIELD


...

........................
La distribución de las categorías para la columna MARITALSTATUS


Married     0.270861
Single      0.202649
Divorced    0.123841
Name: Maritalstatus, dtype: float64

........................
La distribución de las categorías para la columna OVERTIME


No     0.422517
Yes    0.160265
Name: Overtime, dtype: float64

........................


In [50]:
nulls_cat = ['Businesstravel', 'Educationfield', 'Maritalstatus', 'Overtime']

Todos desconocidos!

In [51]:
for columna in nulls_cat:
    # reemplazamos los nulos por el valor Unknown para cada una de las columnas de la lista
    df_rawdata[columna] = df_rawdata[columna].fillna("Unknown")

In [52]:
df_rawdata["Overtime"].unique()

array(['No', 'Unknown', 'Yes'], dtype=object)

# Columna Department: vinculada con Jobrole

In [53]:
# primero vemos cómo está la columna Jobrole
#df_rawdata["Jobrole"].head()

In [54]:
# # capitalize valores de Jobrole (primera letra de cada palabra)
# df_rawdata['Jobrole'] = df_rawdata['Jobrole'].apply(lambda x: ' '.join(word.capitalize() for word in x.lower().split()))
# df_rawdata["Jobrole"].head()

In [55]:
# hacemos función para asignar los valores nulos en Department según la columna Jobrole
def fill_department(df, jobrole, department):
    df.loc[df['Jobrole'] == jobrole, 'Department'] = df.loc[df['Jobrole'] == jobrole, 'Department'].fillna(department)

# Uso de la función fill_department para reemplazar los valores faltantes en 'Department'
fill_department(df_rawdata, 'Research Director', 'Research & Development')
fill_department(df_rawdata, 'Healthcare Representative', 'Research & Development')
fill_department(df_rawdata, 'Laboratory Technician', 'Research & Development')
fill_department(df_rawdata, 'Manufacturing Director', 'Research & Development')
fill_department(df_rawdata, 'Research Scientist', 'Research & Development')
fill_department(df_rawdata, 'Sales Executive', 'Sales')
fill_department(df_rawdata, 'Sales Representative', 'Sales')
fill_department(df_rawdata, 'Human Resources', 'Human Resources')
fill_department(df_rawdata, 'Manager', 'Unknown')

# Mostrar el DataFrame actualizado
df_rawdata[["Jobrole", "Department"]].head(5)

,Jobrole,Department
0,Research Director,NaN
1,Manager,NaN
2,Manager,Research & Development
3,Research Director,NaN
4,Sales Executive,NaN


In [56]:
# Mostrar estadísticas descriptivas y conteo de valores únicos
print("Estadísticas descriptivas para 'Jobrole':\n", df_rawdata['Jobrole'].describe())
print("\nEstadísticas descriptivas para 'Department':\n", df_rawdata['Department'].describe())

print("\nConteo de valores únicos para 'Jobrole':\n", df_rawdata['Jobrole'].value_counts())
print("\nConteo de valores únicos para 'Department':\n", df_rawdata['Department'].value_counts())

Estadísticas descriptivas para 'Jobrole':
 count                  1510
unique                    9
top        Sales Executive 
freq                    336
Name: Jobrole, dtype: object

Estadísticas descriptivas para 'Department':
 count                          283
unique                           3
top        Research & Development 
freq                           185
Name: Department, dtype: object

Conteo de valores únicos para 'Jobrole':
  Sales Executive               336
 Research Scientist            300
 Laboratory Technician         264
 Manufacturing Director        148
 Healthcare Representative     137
 Manager                       105
 Sales Representative           84
 Research Director              83
 Human Resources                53
Name: Jobrole, dtype: int64

Conteo de valores únicos para 'Department':
  Research & Development     185
 Sales                       84
 Human Resources             14
Name: Department, dtype: int64


# Todas las columnas categóricas listas 

In [57]:
# Como hay valores negativos en la columna Distancefromhome los cambiamos a nulos.
def negative_to_null(data):
    if data < 0:
        return np.nan
    else:
        return data
    
df_rawdata["Distancefromhome"] = df_rawdata["Distancefromhome"].apply(negative_to_null)

In [58]:
df_rawdata["Distancefromhome"].tail(5)

1609    3.0
1610    4.0
1611    NaN
1612    8.0
1613    7.0
Name: Distancefromhome, dtype: float64

## Gestión nulos columnas numéricas

In [59]:
nulls_num = df_rawdata[df_rawdata.columns[df_rawdata.isna().any()]].columns
nulls_num

Index(['Dailyrate', 'Department', 'Distancefromhome', 'Monthlyincome',
       'Performancerating', 'Totalworkingyears', 'Worklifebalance'],
      dtype='object')

In [60]:
# Función para quitar "Not Available" y poner nulos en su lugar
def not_available_to_null(data):
    if data == 'Not Available':
        return np.nan
    else:
        return float(data)
df_rawdata["Hourlyrate"] = df_rawdata["Hourlyrate"].apply(not_available_to_null)

In [61]:
df_rawdata_copy = df_rawdata.copy()

In [62]:
print(f"Tenemos nulos: \n{df_rawdata_copy[['Dailyrate', 'Distancefromhome', 'Monthlyincome', 'Performancerating','Totalworkingyears', 'Worklifebalance', 'Hourlyrate']].isnull().sum()} nulos")			

Tenemos nulos: 
Dailyrate            116
Distancefromhome     178
Monthlyincome        799
Performancerating    182
Totalworkingyears    494
Worklifebalance      100
Hourlyrate            80
dtype: int64 nulos


# IterativeImputer

In [63]:
# instanciamos las clases										
imputer_iterative = IterativeImputer(max_iter = 20, random_state = 42)			
# ajustamos y tranformamos los datos										
imputer_iterative_imputado = imputer_iterative.fit_transform(df_rawdata_copy[['Dailyrate', 'Distancefromhome', 'Monthlyincome', 'Performancerating','Totalworkingyears', 'Worklifebalance', 'Hourlyrate']])
# comprobamos que es lo que nos devuelve, que en este caso es un array también					
imputer_iterative_imputado																						

array([[6.84000000e+02, 6.00000000e+00, 1.95370000e+04, ...,
        3.26175277e+01, 3.00000000e+00, 5.10000000e+01],
       [6.99000000e+02, 1.00000000e+00, 1.99990000e+04, ...,
        3.40000000e+01, 3.00000000e+00, 6.50000000e+01],
       [5.32000000e+02, 4.00000000e+00, 1.92320000e+04, ...,
        2.20000000e+01, 2.90522101e+00, 5.80000000e+01],
       ...,
       [9.03000000e+02, 9.05859584e+00, 4.96921235e+03, ...,
        9.00000000e+00, 3.00000000e+00, 4.10000000e+01],
       [1.22900000e+03, 8.00000000e+00, 6.38325083e+03, ...,
        1.20000000e+01, 3.00000000e+00, 8.40000000e+01],
       [5.66000000e+02, 7.00000000e+00, 1.08450000e+04, ...,
        1.84083797e+01, 3.00000000e+00, 7.50000000e+01]])

In [64]:
df_rawdata_copy[['Dailyrate_ITE', 'Distancefromhome_ITE', 'Monthlyincome_ITE', 'Performancerating_ITE','Totalworkingyears_ITE', 'Worklifebalance_ITE', 'Hourlyrate_ITE']] = imputer_iterative_imputado										
										
# comprobamos los nulos										
print(f"Después del 'Iterative' tenemos: \n{df_rawdata_copy[['Dailyrate_ITE', 'Distancefromhome_ITE', 'Monthlyincome_ITE', 'Performancerating_ITE', 'Totalworkingyears_ITE', 'Worklifebalance_ITE', 'Hourlyrate_ITE']].isnull().sum()} nulos")										
										

Después del 'Iterative' tenemos: 
Dailyrate_ITE            0
Distancefromhome_ITE     0
Monthlyincome_ITE        0
Performancerating_ITE    0
Totalworkingyears_ITE    0
Worklifebalance_ITE      0
Hourlyrate_ITE           0
dtype: int64 nulos


# KNN Imputer

In [65]:
# instanciamos la clase del KNNImputer
imputer_knn = KNNImputer(n_neighbors = 5)
# ajustamos y transformamos los datos
imputer_knn_imputado = imputer_knn.fit_transform(df_rawdata_copy[['Dailyrate', 'Distancefromhome', 'Monthlyincome', 'Performancerating','Totalworkingyears', 'Worklifebalance', 'Hourlyrate']])
# comprobamos que es lo que nos devuelve, que sigue siendo un array
imputer_knn_imputado

array([[6.8400e+02, 6.0000e+00, 1.9537e+04, ..., 1.3600e+01, 3.0000e+00,
        5.1000e+01],
       [6.9900e+02, 1.0000e+00, 1.9999e+04, ..., 3.4000e+01, 3.0000e+00,
        6.5000e+01],
       [5.3200e+02, 4.0000e+00, 1.9232e+04, ..., 2.2000e+01, 3.0000e+00,
        5.8000e+01],
       ...,
       [9.0300e+02, 1.0600e+01, 7.1448e+03, ..., 9.0000e+00, 3.0000e+00,
        4.1000e+01],
       [1.2290e+03, 8.0000e+00, 5.8398e+03, ..., 1.2000e+01, 3.0000e+00,
        8.4000e+01],
       [5.6600e+02, 7.0000e+00, 1.0845e+04, ..., 1.5000e+01, 3.0000e+00,
        7.5000e+01]])

In [66]:
# por último nos queda añadir ese array al DataFrame como hemos hecho hasta ahora
df_rawdata_copy[['Dailyrate_KNN', 'Distancefromhome_KNN', 'Monthlyincome_KNN', 'Performancerating_KNN','Totalworkingyears_KNN', 'Worklifebalance_KNN', 'Hourlyrate_KNN']] = imputer_knn_imputado

# comprobamos los nulos
print(f"Después del 'KNN' tenemos: \n{df_rawdata_copy[['Dailyrate_KNN', 'Distancefromhome_KNN', 'Monthlyincome_KNN', 'Performancerating_KNN','Totalworkingyears_KNN', 'Worklifebalance_KNN', 'Hourlyrate_KNN']].isnull().sum()} nulos")


Después del 'KNN' tenemos: 
Dailyrate_KNN            0
Distancefromhome_KNN     0
Monthlyincome_KNN        0
Performancerating_KNN    0
Totalworkingyears_KNN    0
Worklifebalance_KNN      0
Hourlyrate_KNN           0
dtype: int64 nulos


In [29]:
df_rawdata_copy.describe()[['Dailyrate', 'Dailyrate_ITE', 'Dailyrate_KNN']]

,Dailyrate,Dailyrate_ITE,Dailyrate_KNN
count,1394.000000,1510.000000,1510.000000
mean,807.050215,807.810811,807.108742
std,401.842182,386.574125,388.697789
min,103.000000,103.000000,103.000000
25%,472.500000,504.250000,501.000000
50%,805.500000,807.000000,806.500000
75%,1157.750000,1135.500000,1137.000000
max,1499.000000,1499.000000,1499.000000


In [30]:
df_rawdata_copy.describe()[["Distancefromhome", "Distancefromhome_ITE", "Distancefromhome_KNN"]]

,Distancefromhome,Distancefromhome_ITE,Distancefromhome_KNN
count,1332.000000,1510.000000,1510.000000
mean,8.960961,8.964497,9.147682
std,7.954886,7.471850,7.582841
min,1.000000,1.000000,1.000000
25%,2.000000,2.000000,2.000000
50%,7.000000,8.000000,7.500000
75%,13.000000,11.000000,13.000000
max,29.000000,29.000000,29.000000


In [31]:
df_rawdata_copy.describe()[["Monthlyincome", "Monthlyincome_ITE", "Monthlyincome_KNN"]]

,Monthlyincome,Monthlyincome_ITE,Monthlyincome_KNN
count,711.000000,1510.000000,1510.000000
mean,6527.918425,6379.088831,6352.467285
std,4810.541121,4042.244826,3550.743814
min,1009.000000,-332.052371,1009.000000
25%,2906.500000,3508.857859,4250.750000
50%,4850.000000,5492.033242,5558.600000
75%,8456.000000,7376.777075,7312.550000
max,19999.000000,20965.060484,19999.000000


In [32]:
df_rawdata_copy.describe()[["Performancerating", "Performancerating_ITE", "Performancerating_KNN"]]

,Performancerating,Performancerating_ITE,Performancerating_KNN
count,1328.000000,1510.000000,1510.000000
mean,3.155120,3.155168,3.154570
std,0.362156,0.339629,0.343262
min,3.000000,3.000000,3.000000
25%,3.000000,3.000000,3.000000
50%,3.000000,3.000000,3.000000
75%,3.000000,3.140821,3.000000
max,4.000000,4.000000,4.000000


In [33]:
df_rawdata_copy.describe()[["Totalworkingyears","Totalworkingyears_ITE", "Totalworkingyears_KNN"]]

,Totalworkingyears,Totalworkingyears_ITE,Totalworkingyears_KNN
count,1016.000000,1510.000000,1510.000000
mean,11.314961,11.555397,12.087020
std,7.794100,7.224061,6.972891
min,0.000000,0.000000,0.000000
25%,6.000000,6.961697,7.200000
50%,10.000000,10.000000,10.400000
75%,15.000000,13.981491,16.000000
max,40.000000,40.000000,40.000000


In [34]:
df_rawdata_copy.describe()[["Worklifebalance", "Worklifebalance_ITE", "Worklifebalance_KNN"]]

,Worklifebalance,Worklifebalance_ITE,Worklifebalance_KNN
count,1410.000000,1510.000000,1510.000000
mean,2.756028,2.755626,2.730331
std,0.705888,0.682205,0.696384
min,1.000000,1.000000,1.000000
25%,2.000000,2.000000,2.000000
50%,3.000000,3.000000,3.000000
75%,3.000000,3.000000,3.000000
max,4.000000,4.000000,4.000000


In [67]:
df_rawdata_copy.describe()[["Hourlyrate","Hourlyrate_ITE","Hourlyrate_KNN"]]

,Hourlyrate,Hourlyrate_ITE,Hourlyrate_KNN
count,1430.000000,1510.000000,1510.000000
mean,66.025175,66.025926,65.930728
std,20.270996,19.727124,19.792448
min,30.000000,30.000000,30.000000
25%,48.000000,49.000000,49.000000
50%,66.000000,66.000000,66.000000
75%,84.000000,83.000000,83.000000
max,100.000000,100.000000,100.000000


Dailyrate_KNN, Distancefromhome_KNN, Monthlyincome_KNN, Performancerating_KNN, Totalworkingyears_KNN, Worklifebalance_KNN, Hourlyrate_KNN

# Hemos seleccionado el método KNN Imputer ya que los resultados son más precisos.


In [68]:
#al final me quedo con las del método kn por escoger alguna
df_rawdata_copy.drop(['Dailyrate', 'Dailyrate_ITE', 'Distancefromhome', 'Distancefromhome_ITE', 'Monthlyincome', 'Monthlyincome_ITE', 'Performancerating', 'Performancerating_ITE', 'Totalworkingyears', 'Totalworkingyears_ITE', 'Worklifebalance', 'Worklifebalance_ITE', 'Hourlyrate', 'Hourlyrate_ITE'], axis = 1, inplace = True)

# ahora vamos a cambiar el nombre de las columnas que quedaron para que tengan el mismo nombre de origen
nuevo_nombre = {"Dailyrate_KNN": "Dailyrate",  'Distancefromhome_KNN': "Distancefromhome", "Monthlyincome_KNN": "Monthlyincome", "Performancerating_KNN": "Performancerating","Totalworkingyears_KNN": "Totalworkingyears", "Worklifebalance_KNN": "Worklifebalance", "Hourlyrate_KNN":"Hourlyrate"}
df_rawdata_copy.rename(columns = nuevo_nombre, inplace = True)


In [69]:
df_rawdata_copy.columns

Index(['Age', 'Attrition', 'Businesstravel', 'Department', 'Education',
       'Educationfield', 'Employeenumber', 'Environmentsatisfaction', 'Gender',
       'Jobinvolvement', 'Joblevel', 'Jobrole', 'Jobsatisfaction',
       'Maritalstatus', 'Monthlyrate', 'Numcompaniesworked', 'Overtime',
       'Percentsalaryhike', 'Relationshipsatisfaction', 'Stockoptionlevel',
       'Trainingtimeslastyear', 'Yearsatcompany', 'Yearssincelastpromotion',
       'Yearswithcurrmanager', 'Datebirth', 'Remotework', 'Dailyrate',
       'Distancefromhome', 'Monthlyincome', 'Performancerating',
       'Totalworkingyears', 'Worklifebalance', 'Hourlyrate'],
      dtype='object')

In [71]:
df_rawdata_copy.to_csv("raw_data_final_20231205.csv")